In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cyberbullying-dataset/twitter_sexism_parsed_dataset.csv
/kaggle/input/cyberbullying-dataset/youtube_parsed_dataset.csv
/kaggle/input/cyberbullying-dataset/kaggle_parsed_dataset.csv
/kaggle/input/cyberbullying-dataset/aggression_parsed_dataset.csv
/kaggle/input/cyberbullying-dataset/toxicity_parsed_dataset.csv
/kaggle/input/cyberbullying-dataset/attack_parsed_dataset.csv
/kaggle/input/cyberbullying-dataset/twitter_parsed_dataset.csv
/kaggle/input/cyberbullying-dataset/twitter_racism_parsed_dataset.csv


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer
import tqdm
import pandas as pd

In [3]:
aggression_df = pd.read_csv("/kaggle/input/cyberbullying-dataset/aggression_parsed_dataset.csv")
attack_df = pd.read_csv("/kaggle/input/cyberbullying-dataset/attack_parsed_dataset.csv")
kaggle_df = pd.read_csv("/kaggle/input/cyberbullying-dataset/kaggle_parsed_dataset.csv")
toxicity_df = pd.read_csv("/kaggle/input/cyberbullying-dataset/toxicity_parsed_dataset.csv")
twitter_parsed_df = pd.read_csv("/kaggle/input/cyberbullying-dataset/twitter_parsed_dataset.csv")
twitter_racism_df = pd.read_csv("/kaggle/input/cyberbullying-dataset/twitter_racism_parsed_dataset.csv")
twitter_sexism_df  = pd.read_csv("/kaggle/input/cyberbullying-dataset/twitter_sexism_parsed_dataset.csv")
youtube_df = pd.read_csv("/kaggle/input/cyberbullying-dataset/youtube_parsed_dataset.csv")

In [4]:
count_ones = aggression_df['oh_label'][aggression_df['oh_label'] == 1].value_counts()
count_zeros =  aggression_df['oh_label'][aggression_df['oh_label'] == 0].value_counts()
print(count_ones)
print("---")
print(count_zeros)

oh_label
1    14782
Name: count, dtype: int64
---
oh_label
0    101082
Name: count, dtype: int64


In [5]:
# Concatonate dataframes by extracting relevant columns
df = [
    aggression_df[["Text", "oh_label"]], 
    attack_df[["Text", "oh_label"]],
    kaggle_df[["Text", "oh_label"]],
    toxicity_df[["Text", "oh_label"]],
    twitter_parsed_df[["Text", "oh_label"]],
    twitter_racism_df[["Text", "oh_label"]],
    twitter_sexism_df[["Text", "oh_label"]],
    youtube_df[["Text", "oh_label"]]
]

comb_df = pd.concat(df, ignore_index=True)

In [6]:
count_ones = comb_df['oh_label'][comb_df['oh_label'] == 1].value_counts()
count_zeros = comb_df['oh_label'][comb_df['oh_label'] == 0].value_counts()
print(count_ones)
print("---")
print(count_zeros)

oh_label
1.0    57651
Name: count, dtype: int64
---
oh_label
0.0    391223
Name: count, dtype: int64


In [7]:
required_sent_num_0 = 15_000
required_sent_num_1 = 15_000

oh_label_0_sample = comb_df[comb_df['oh_label'] == 0].sample(n=required_sent_num_0, random_state=42)
oh_label_1_sample = comb_df[comb_df['oh_label'] == 1].sample(n=required_sent_num_1, random_state=42)

# Combine the two subsets
prefered_df = pd.concat([oh_label_0_sample, oh_label_1_sample], ignore_index=True)

# Shuffle the resulting dataframe 
prefered_df = prefered_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
prefered_df

,Text,oh_label
0,:Believe the protection had better stay indef...,0.0
1,@Qoloob4 @Vandaliser @sajid_fairooz @IsraeliRe...,1.0
2,RT @pdemmett: Why do film bosses have to make ...,1.0
3,You piece of shit fucking block me then puss...,1.0
4,==SimCopter Shenanigans== Hiya. I originall...,0.0
...,...,...
29995,== Fuck you == Fuck you shit asshole! You a...,1.0
29996,Yeah his headstock is just upside down,0.0
29997,you are correct. If you look at who created ...,0.0
29998,"Also, my mother is a prostitute.",1.0


In [9]:
count_ones = prefered_df['oh_label'][prefered_df['oh_label']==1].count()
count_zeros = prefered_df['oh_label'][prefered_df['oh_label']==0].count()

print(count_ones, count_zeros)

15000 15000


In [10]:
# Clean the Text
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove mentions and hashtags
    text = re.sub(r'\@\w+|\#', '', text)
    # Remove contractions and special characters
    pattern = r"'s|'t|'re|'ve|'m|'ll|'d|[^a-zA-Z0-9\s]+|\s+"
    text = re.sub(pattern, " ", text)
    return text.lower().strip()

prefered_df['Cleaned_text'] = prefered_df['Text'].apply(clean_text) 

In [11]:
prefered_df

,Text,oh_label,Cleaned_text
0,:Believe the protection had better stay indef...,0.0,believe the protection had better stay indefin...
1,@Qoloob4 @Vandaliser @sajid_fairooz @IsraeliRe...,1.0,which is why he gave himself as many wives as ...
2,RT @pdemmett: Why do film bosses have to make ...,1.0,rt why do film bosses have to make life weir...
3,You piece of shit fucking block me then puss...,1.0,you piece of shit fucking block me then pussy ...
4,==SimCopter Shenanigans== Hiya. I originall...,0.0,simcopter shenanigans hiya i originally foll...
...,...,...,...
29995,== Fuck you == Fuck you shit asshole! You a...,1.0,fuck you fuck you shit asshole you are a pi...
29996,Yeah his headstock is just upside down,0.0,yeah his headstock is just upside down
29997,you are correct. If you look at who created ...,0.0,you are correct if you look at who created th...
29998,"Also, my mother is a prostitute.",1.0,also my mother is a prostitute


In [12]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    prefered_df['Cleaned_text'], prefered_df['oh_label'], test_size=0.2,random_state=42)

In [13]:
train_texts.head()

21753    what identity is that  dracula  why is it that...
251      dude  i do it because your mum screams tooooo ...
22941    windows product activation suggestions   hi  i...
618      i  only talking about this quote  although oth...
17090    strategy toolkit  strategy toolkit has been no...
Name: Cleaned_text, dtype: object

In [14]:
from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Assign a padding token
tokenizer.pad_token = tokenizer.eos_token 

# Tokenize the Texts
train_encodings = tokenizer(
    list(train_texts), 
    truncation=True,
    padding=True, 
    max_length=256, 
    return_tensors='pt'
)

val_encodings = tokenizer(
    list(val_texts), 
    truncation=True,
    padding=True, 
    max_length=256, 
    return_tensors='pt'
)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
from torch.utils.data import Dataset, DataLoader

In [16]:
# Creating Custom datasets

class SentimentAnalysis(Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        # Fetch the encoding for the given index
        item = {key: val[idx].clone() for key, val in self.encodings.items()}
        # Fetch the label for the given index
        label = torch.tensor(self.labels[idx])  # Convert the label to a tensor
        return item, label


In [17]:
# Create dataset 
train_data = SentimentAnalysis(train_encodings, list(train_labels))
val_data = SentimentAnalysis(val_encodings, list(val_labels))

In [18]:
import torch.nn as nn
from transformers import GPT2Config, GPT2Model

# Load GPT-2 configuration
config = GPT2Config.from_pretrained('gpt2')
config.num_labels = 2  # For binary classification

# Define the custom model class
class GPTForSentiment(nn.Module):
    def __init__(self, config):
        super(GPTForSentiment, self).__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2', config=config)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)  # Classification Layer
        
    def forward(self, input_ids, attention_mask):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        # Use the last hidden state from the last token for classification
        logits = self.classifier(outputs.last_hidden_state[:, -1, :])
        return logits

In [19]:
model = GPTForSentiment(config)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [20]:
# set up dataloaders
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=8)

# Model Parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-05)

In [21]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

GPTForSentiment(
  (gpt2): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (classifier): Linear(in_features=768, out_features=2, bias=True)
)

In [24]:
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm import tqdm

In [25]:
# Training method and loop
def train_fn(train_dataloader, epochs, model, loss_fn, optimizer, device):
    model.to(device)  # Move model to device (CPU or GPU)

    for epoch in range(epochs):  # Fixed missing closing parenthesis
        model.train()  # Ensure the model is in training mode
        
        loop = tqdm(train_dataloader, leave=True)
        all_preds_train = []
        all_labels_train = []
        all_preds_labels_train = []  # To store predicted labels for accuracy calculation
        
        for batch in loop: 
            
            inputs, labels = batch
            # Move inputs and labels to the device
            if isinstance(inputs, dict):
                inputs = {key: val.to(device) for key, val in inputs.items()}
                attention_mask = inputs.get('attention_mask')  # Extract the attention mask if present
            else: 
                inputs = inputs.to(device)
                attention_mask = None  # No attention mask if not present
            labels = labels.to(device)
            labels = labels.long()

            optimizer.zero_grad()
            
            # Pass attention_mask only if it exists
            if attention_mask is not None:
                outputs = model(inputs['input_ids'], attention_mask=attention_mask)
            else:
                outputs = model(inputs)
                
            # Compute the loss
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Convert logits to probabilities (if applicable)
            probs = torch.softmax(outputs, dim=1)
            preds = probs[:, 1]  # Get probability for the positive class
            
            # Convert probabilities to class predictions (0 or 1)
            preds_class = (preds > 0.5).long()  # Threshold at 0.5
            
            # Store predictions and labels for AUC and accuracy
            all_preds_train.extend(preds.detach().cpu().numpy())
            all_labels_train.extend(labels.cpu().numpy())
            all_preds_labels_train.extend(preds_class.cpu().numpy())
            
            # Update progress bar
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(loss=loss.item())
        
        # Calculate training AUC after the epoch
        train_auc = roc_auc_score(all_labels_train, all_preds_train)
        train_accuracy = accuracy_score(all_labels_train, all_preds_labels_train)  # Calculate accuracy
        print(f"Epoch {epoch} - Training AUC: {train_auc:.4f}, Training Accuracy: {train_accuracy:.4f}")


In [26]:
train_fn(train_loader, 6, model, loss_fn, optimizer, device)

Epoch 0: 100%|██████████| 3000/3000 [21:03<00:00,  2.37it/s, loss=0.24]   


Epoch 0 - Training AUC: 0.9382, Training Accuracy: 0.8654


Epoch 1: 100%|██████████| 3000/3000 [21:07<00:00,  2.37it/s, loss=0.35]   


Epoch 1 - Training AUC: 0.9686, Training Accuracy: 0.9118


Epoch 2: 100%|██████████| 3000/3000 [21:08<00:00,  2.37it/s, loss=0.0967]  


...

Epoch 3 - Training AUC: 0.9931, Training Accuracy: 0.9656


Epoch 4: 100%|██████████| 3000/3000 [21:08<00:00,  2.37it/s, loss=0.0615]  


Epoch 4 - Training AUC: 0.9966, Training Accuracy: 0.9779


Epoch 5: 100%|██████████| 3000/3000 [21:08<00:00,  2.36it/s, loss=0.003]   

Epoch 5 - Training AUC: 0.9981, Training Accuracy: 0.9826


In [27]:
# Validation Function
def validate_fn(val_dataloader, model):
    model.eval()  # Ensure the model is in evaluation mode
    
    all_preds_val = []
    all_labels_val = []
    with torch.no_grad():  # Disable gradient calculations for validation
        for batch in val_dataloader:
            inputs, labels = batch
            
            # Move inputs and labels to device
            if isinstance(inputs, dict):  # For tokenized inputs
                inputs = {key: val.to(device) for key, val in inputs.items()}
                attention_mask = inputs.get('attention_mask')  # Extract attention_mask if present
            else:  # For simple tensors
                inputs = inputs.to(device)
                attention_mask = None  # No attention mask available
            labels = labels.to(device)

            # Pass attention_mask only if it exists
            if attention_mask is not None:
                outputs = model(inputs['input_ids'], attention_mask=attention_mask)
            else:
                outputs = model(inputs)
            
            # Convert logits to probabilities (if applicable)
            probs = torch.softmax(outputs, dim=1)
            preds = probs[:, 1]  # Get probability for the positive class
            
            # Store for AUC
            all_preds_val.extend(preds.cpu().numpy())
            all_labels_val.extend(labels.cpu().numpy())
    # Calculate validation AUC
    val_auc = roc_auc_score(all_labels_val, all_preds_val)
    print(f"Validation AUC: {val_auc:.4f}")

In [28]:
validate_fn(val_loader, model)

Validation AUC: 0.9586


In [29]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")  # Use the correct tokenizer
# Add a custom pad token if not already defined
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Update the model's tokenizer settings
tokenizer.pad_token = '[PAD]'

def preprocess_input(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return inputs


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [30]:
import torch.nn.functional as F

def test_sentence(sentence):
    # Preprocess the input sentence
    inputs = preprocess_input(sentence)

    # Move inputs to the same device as the model (if you are using a GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    model.to(device)

    # Run the sentence through the model
    with torch.no_grad():  # Disable gradients for inference
        outputs = model(**inputs)
    print(outputs)

    # Get the logits (raw predictions)
    logits = outputs

    # Convert logits to probabilities using softmax
    probabilities = F.softmax(logits, dim=1)

    # Get predicted class (1 for cyberbullying, 0 for non-cyberbullying)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    # Print the result
    if predicted_class == 0:
        print(f"The sentence is classified as: Not Cyberbullying")
    else:
        print(f"The sentence is classified as: Cyberbullying")

    return predicted_class, probabilities


In [43]:
sentence = "You must have small penis"

predicted_class, probabilities = test_sentence(sentence)

# Output the predicted class and probabilities
print(f"Predicted Class: {predicted_class}")  # 0 = Non-Cyberbullying, 1 = Cyberbullying
print(f"Probabilities: {probabilities}")

tensor([[-0.1155,  0.0071]], device='cuda:0')
The sentence is classified as: Cyberbullying
Predicted Class: 1
Probabilities: tensor([[0.4694, 0.5306]], device='cuda:0')
